In [1]:
import river
import river.datasets as datasets
from river import stream
from pprint import pprint
import os
from scipy.io import arff
import pandas as pd
from river import tree
from river import evaluate
from river import metrics

import numpy as np
from  streams.stream_section import StreamSection
from streams.utils import FL, FU, generate_stream_section
from my_datasets.getting_datasets import get_LED,get_Hyperplane,get_Electricity,get_AGRAWL,get_Airlines,get_CoverType,get_RandomRBF,transform_Electricity
from train_and_eval import train_and_evaluate
from functools import partial
from train_and_eval import train_and_evaluate
from river.metrics.accuracy import Accuracy
from river.metrics import CohenKappa
from multiprocessing.pool import ThreadPool
from river.datasets.synth import ConceptDriftStream
from tqdm import tqdm
import pickle

# methods
from river.dummy import NoChangeClassifier,PriorClassifier
from river.cluster import CluStream
from river.forest.adaptive_random_forest import ARFClassifier
from river.tree.hoeffding_adaptive_tree_classifier import  HoeffdingAdaptiveTreeClassifier
from river.neighbors import KNNClassifier as riverKNN
from river.naive_bayes import GaussianNB
from semisupervised_methods.clustream_and_label import CluserAndLabel
from semisupervised_methods.incremental_classifier import IncrementalClassifer
from semisupervised_methods.one_nearest_neighbour import oneNNClassifer
from river.drift import ADWIN, PageHinkley
from semisupervised_methods.ICLC import ICLC
from river.cluster import KMeans


# Classifers


# Notebook

In [2]:
# initial = generate_stream_section(get_CoverType(),'Cover_Type','Cover_Type',stop = 20000)

In [3]:
import logging
import datetime
now = datetime.datetime.now()
date_time = now.strftime("%d_%m_%Y_%H_%M_%S")
logging.basicConfig(filename=f'logs\\{date_time}.log', filemode='w',
                    format='%(asctime)s - %(message)s', level=logging.INFO, datefmt='%d-%b-%y %H:%M:%S')
logger = logging.getLogger()

In [4]:
# get the data stream
electricity = get_Electricity()
initial_stream = transform_Electricity(electricity)
Q=[(0,20000)]
probas = [0.1,0.2]  # probabilities of dropping the label
HT1 = HoeffdingAdaptiveTreeClassifier
ARF1 = ARFClassifier
NB1= GaussianNB
incClasif2= IncrementalClassifer
incClasif1= IncrementalClassifer
incClasif3= IncrementalClassifer
m_params = [

    {'threshold':0.7,
  'classifier':NB1,
  'params':{}}
]
delay_type =0
methods = [incClasif1,incClasif2]
res = train_and_evaluate(initial_stream=initial_stream,  Q=Q,
                         probas=probas, methods=methods, methods_params=m_params,
                         methods_name=['NB07'],
                         metric_fun=Accuracy, delay_type=delay_type, B=50, K=10, delay=1000,logger = logger, warm_up_period=480)


100%|██████████| 45312/45312 [00:00<00:00, 335266.12it/s]


[{'threshold': 0.7, 'classifier': <class 'river.naive_bayes.gaussian.GaussianNB'>, 'params': {}}][{'threshold': 0.7, 'classifier': <class 'river.naive_bayes.gaussian.GaussianNB'>, 'params': {}}]

[{'threshold': 0.7, 'classifier': <class 'river.naive_bayes.gaussian.GaussianNB'>, 'params': {}}]
[{'threshold': 0.7, 'classifier': <class 'river.naive_bayes.gaussian.GaussianNB'>, 'params': {}}]
list index out of range 0
[{'threshold': 0.7, 'classifier': <class 'river.naive_bayes.gaussian.GaussianNB'>, 'params': {}}]
list index out of range 0
[{'threshold': 0.7, 'classifier': <class 'river.naive_bayes.gaussian.GaussianNB'>, 'params': {}}]
list index out of range 0


IndexError: list index out of range

In [4]:
# get the data stream
electricity = get_Electricity()
initial_stream = transform_Electricity(electricity)
Q=[(0,20000)]
probas = [0.1,0.2]  # probabilities of dropping the label
clustream = CluserAndLabel

m_params = [
    {'seed':123}]
methods = [clustream]
delay_type = 0 # constant delay

# # run the experiment 
res = train_and_evaluate(initial_stream=initial_stream,  Q=Q,
                         probas=probas, methods=methods, methods_params=m_params,
                         methods_name=['CluStream'],
                         metric_fun=Accuracy, delay_type=delay_type, B=50, K=10, delay=1000,logger = logger, warm_up_period=480)


100%|██████████| 45312/45312 [00:00<00:00, 499093.48it/s]


Electricity_0_20000
Electricity_0_20000_ssl_0.1_0_20000
Electricity_0_20000_ssl_0.2_0_20000


In [6]:
electricity = get_Electricity()
initial_stream = transform_Electricity(electricity)
Q=[(0,20000)]
probas = [0.1,0.2]  # probabilities of dropping the label
my_method = ICLC
Cl1= KMeans
ARF = ARFClassifier
m_params = [
 {  'classifer':ARF,
  'clustering_method':Cl1,
  'clustering_params':{},
  'classifier_params':{},
  'drift_detector': ADWIN,
  'nu':100}
]
methods = [my_method]
delay_type = 0 # constant delay

# # run the experiment 
res = train_and_evaluate(initial_stream=initial_stream,  Q=Q,
                         probas=probas, methods=methods, methods_params=m_params,
                         methods_name=['ICLC'],
                         metric_fun=CohenKappa, delay_type=delay_type, B=50, K=10, delay=1000,logger = logger, warm_up_period=480)


100%|██████████| 45312/45312 [00:00<00:00, 166839.14it/s]


Electricity_0_20000
Electricity_0_20000_ssl_0.1_0_20000
Electricity_0_20000_ssl_0.2_0_20000


In [ ]:
electricity = get_Electricity()
initial_stream = transform_Electricity(electricity)
Q=[(0,20000)]
probas = [0.1,0.2]  # probabilities of dropping the label
my_method = ICLC
Cl1= KMeans
ARF = ARFClassifier
m_params = [
 {  'classifer':ARF,
  'clustering_method':Cl1,
  'clustering_params':{},
  'classifier_params':{},
  'drift_detector': ADWIN,
  'nu':100}
]
methods = [my_method]
delay_type = 1 # constant delay

# # run the experiment 
res = train_and_evaluate(initial_stream=initial_stream,  Q=Q,
                         probas=probas, methods=methods, methods_params=m_params,
                         methods_name=['ICLC'],
                         metric_fun=Accuracy, delay_type=delay_type, B=50, K=10, delay=1000,logger = logger)


In [4]:
logger = logging.getLogger()

In [6]:
ag1 =  get_AGRAWL( balance_classes= True, perturbation=0.3)
ag2 =  get_AGRAWL( balance_classes= True, perturbation=0.6)
stream = ConceptDriftStream(ag1,ag2, position=10000,width=20000)
initial_stream = generate_stream_section(stream,'AGRAWAL','synth',start=0,stop=20000)
Q=[(0,len(initial_stream.stream))]
probas = [0.1,0.2]  # probabilities of dropping the label
dummy1 = PriorClassifier
dummy2 = NoChangeClassifier
incClasif2= IncrementalClassifer
incClasif1= IncrementalClassifer
m_params = [

  {'threshold':0.7,
  'classifier':dummy2,
  'params':{}}
]
delay_type =1
methods = [incClasif1]
res = train_and_evaluate(initial_stream=initial_stream,  Q=Q,
                         probas=probas, methods=methods, methods_params=m_params,
                         methods_name=['Majority'],
                         metric_fun=Accuracy, delay_type=delay_type, B=50, K=10, delay=1000,logger = logger)

AGRAWAL_0_20000
AGRAWAL_0_20000_constant_delay
AGRAWAL_0_20000_constant_delay_ssl_0.1_0_20000
AGRAWAL_0_20000_constant_delay_ssl_0.2_0_20000


In [ ]:
import logging
import datetime
now = datetime.datetime.now()
date_time = now.strftime("%d_%m_%Y_%H_%M_%S")
logging.basicConfig(filename=f'logs\\{date_time}.log', filemode='w',
                    format='%(asctime)s - %(message)s', level=logging.INFO, datefmt='%d-%b-%y %H:%M:%S')
logger = logging.getLogger()

In [6]:
# get the data stream
led3 = get_LED(0.1,True,3)
led5 = get_LED(0.1,True,5)
led7 = get_LED(0.1,True,7)

cds = ConceptDriftStream(ConceptDriftStream(led3,led5,position=3750,width = 3750),ConceptDriftStream(led5,led7,position=7500,width = 5000), position=7500,width=7500)
initial_stream = generate_stream_section(cds,'LED_gradual','synth',start=0,stop=20000)
Q=[(0,20000)]
probas = [0.1,0.2]  # probabilities of dropping the label
dummy1 = PriorClassifier
dummy2 = NoChangeClassifier
incClasif2= IncrementalClassifer
incClasif1= IncrementalClassifer
m_params = [

  {'threshold':0.7,
  'classifier':dummy1,
  'params':{}},
  {'threshold':0.7,
  'classifier':dummy2,
  'params':{}}
]
delay_type =1
methods = [incClasif1,incClasif2]
res = train_and_evaluate(initial_stream=initial_stream,  Q=Q,
                         probas=probas, methods=methods, methods_params=m_params,
                         methods_name=['Prior','Majority'],
                         metric_fun=Accuracy, delay_type=delay_type, B=50, K=10, delay=1000,logger = logger)


In [7]:
# get the data stream
led3 = get_LED(0.1,True,3)
led5 = get_LED(0.1,True,5)
led7 = get_LED(0.1,True,7)

st = [(cur_idx, cur_idx, instance[0], instance[1]) for cur_idx, instance in enumerate(led3.take(3750))]
st.extend([(cur_idx, cur_idx, instance[0], instance[1]) for cur_idx, instance in enumerate(led5.take(7500))])
st.extend([(cur_idx, cur_idx, instance[0], instance[1]) for cur_idx, instance in enumerate(led7.take(8750))])
initial_stream = StreamSection('LED_abrupt', st, True)
Q=[(0,20000)]
probas = [0.1,0.2]  # probabilities of dropping the label
dummy1 = PriorClassifier
dummy2 = NoChangeClassifier
incClasif2= IncrementalClassifer
incClasif1= IncrementalClassifer
m_params = [

  {'threshold':0.7,
  'classifier':dummy1,
  'params':{}},
  {'threshold':0.7,
  'classifier':dummy2,
  'params':{}}
]
delay_type =1
methods = [incClasif1,incClasif2]
res = train_and_evaluate(initial_stream=initial_stream,  Q=Q,
                         probas=probas, methods=methods, methods_params=m_params,
                         methods_name=['Prior','Majority'],
                         metric_fun=Accuracy, delay_type=delay_type, B=50, K=10, delay=1000,logger = logger)


In [9]:
# get the data stream
rbf = get_RandomRBF(change_speed=0.0001, n_centroids=50)

# generate stream secion from it
initial_stream = generate_stream_section(datastream=rbf, stream_name='RBF_moderate',
                                         dataset_type='synth', start=0, stop=2000)
Q=[(0,20000)]
probas = [0.1,0.2]  # probabilities of dropping the label
dummy1 = PriorClassifier
dummy2 = NoChangeClassifier
incClasif2= IncrementalClassifer
incClasif1= IncrementalClassifer
m_params = [

  {'threshold':0.7,
  'classifier':dummy1,
  'params':{}},
  {'threshold':0.7,
  'classifier':dummy2,
  'params':{}}
]
delay_type =1
methods = [incClasif1,incClasif2]
res = train_and_evaluate(initial_stream=initial_stream,  Q=Q,
                         probas=probas, methods=methods, methods_params=m_params,
                         methods_name=['Prior','Majority'],
                         metric_fun=Accuracy, delay_type=delay_type, B=50, K=10, delay=1000,logger = logger)


In [5]:
# save dictionary to person_data.pkl file
with open(f'distribution\\CoverType_mymethod.pkl', 'wb') as fp:
    pickle.dump(res, fp)
    print('dictionary saved successfully to file')

dictionary saved successfully to file


In [ ]:
# save dictionary to person_data.pkl file
with open(f'distribution\\LED_gradual1.pkl', 'wb') as fp:
    pickle.dump(res, fp)
    print('dictionary saved successfully to file')

dictionary saved successfully to file


# DATASETS

In [ ]:
fligths = [(dict((i,x[i]) for i in x.keys() if i!='Delay'),x['Delay']) for x,_ in stream.iter_arff('../../datasets/airlines.arff.zip', compression='infer')]
lda = datasets.synth.LEDDrift(123,0,True,0)
electricity = [(dict((i,x[i]) for i in x.keys() if i!='class'),x['class']=='UP') for x,_ in stream.iter_arff('../../datasets/elecNormNew.arff.zip', compression='infer')]